## This Notebook Goes over the phase analysis functions

In [ ]:
%reload_ext autoreload
%autoreload 2
import phasecheck as pc
import sxmreader as sxm
import trackpy as tp
import matplotlib.pyplot as plt
from PIL import Image
from sxmreader import SXMReader
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
from phasecheck import get_cffits

### The next two cells are quick checks to see if data can load correctly

In [ ]:
#Getting data with SXM

path=['test_images/rehexaticphase/Image_233.sxm']
img = SXMReader(path, correct='lines')[0]
_kwargs={
    'diameter':5,
    'maxsize':200,
    'minmass':1,
    'separation':1,}
fs = tp.locate(img,**_kwargs,engine='python') #gets data
tp.annotate(fs,img) #plots image with the red circles


In [ ]:
#Getting data with jpg/png
path=['test_images/Image_924 Image Z (m).jpg']
_kwargs={
    'diameter':5,
    'maxsize':200,
    'minmass':1,
    'separation':2,}
img = Image.open(path[0])
#This part just to get img readable for trackpy
img = img.resize((200,200),)
img=img.convert('L')
img.ndim=2
img.shape=img.size

fs=tp.locate(img,**_kwargs).reset_index()
tp.annotate(fs,img)


### Unified into a class called imgdata from phasecheck.py

In [ ]:
#The two cells above are esentially done when initializing a 
#phasecheck.imgdata instance

_kwargs={
    'diameter':5,
    'maxsize':300,
    'minmass':2,
    'separation':1,
    
    }

path=['test_images/rehexaticphase/Image_723.sxm']
img_d=pc.imgdata(path,filetype='SXM',**_kwargs)
plt.imshow(img_d.img)
plt.show()
tp.annotate(img_d.data,img_d.img)



### Useful functions

In [ ]:
#SXM files have scan dimensions in meters
xdim=SXMReader(path).scans[0].size['real']['x']*100 
ydim=SXMReader(path).scans[0].size['real']['y']*100

density=img_d.data.shape[0]/(xdim*ydim) #calculate density
print(f'density={density:E}')

In [ ]:
#plots Voronoi diagram, kind=0 for nearest neighbor
pc.plot_boops(img_d,kind=0,realdim=True,) 


In [ ]:

#kind=1 for orientation
pc.plot_boops(img_d,kind=1,realdim=True,) 


In [ ]:
#orientational order 
#start and dist choose starting point and distance between peaks,
#fit_id=None for best fit, 0,1,2 to choose what curve
#periodic chooses if we want the image to have a periodic boundary: generally should be false
pc.plot_bocf(img_d,start=1,fit_id=None,realdim=True,p_dist=5,yscale_plot='log',periodic=False,)


In [ ]:
#translational order 
#Translational order requires a reciprocal vector G. This is chosen by finding a peak in the diffraction pattern, marked with an x.
pc.plot_tocf(img_d,realdim=True)

In [ ]:







#pc.plot_rdf(img_d,start=1,fit_id=None)


#structure factor
pc.plot_ssf2(img_d,upper=0.8,lower=0.001,grid_size=512,dpi=400,realdim=True,)
